## Which DNA patterns serve as a molecular clock?

The daily schedules of animals, plants, and even bacteria are controlled by an internal timekeeper called the circadian clock. Anyone who has experienced the misery of jet lag knows that this clock never stops ticking. Rats and research volunteers alike, when placed in a bunker, naturally maintain a roughly 24-hour cycle of activity and rest in total darkness. And, like any timepiece, the circadian clock can malfunction, resulting in a genetic disease known as delayed sleep-phase syndrome (DSPS).

The circadian clock must have some basis on the molecular level, which presents many questions. How do individual cells in animals and plants, let alone bacteria, know when they should slow down or increase the production of certain proteins? Is there a “clock gene”? Can we explain why heart attacks occur more often in the morning, while asthma attacks are more common at night? And can we identify genes that are responsible for “breaking” the circadian clock to cause DSPS?

In the early 1970s, Ron Konopka and Seymour Benzer identified mutant flies with abnormal circadian patterns and traced the flies’ mutations to a single gene. Biologists needed two more decades to discover a similar clock gene in mammals, which was just the first piece of the puzzle. Today, many more circadian genes have been discovered; these genes, having names like timeless, clock, and cycle, orchestrate the behavior of hundreds of other genes and display a high degree of evolutionary conservation across species.

We will first focus on plants, since maintaining the circadian clock in plants is a matter of life and death. Consider how many plant genes should pay attention to the time when the sun rises and sets; indeed, biologists estimate that over a thousand plant genes are circadian, including the genes related to photosynthesis, photo reception, and flowering. These genes must somehow know what time it is in order to change their gene transcript production, or gene expression, throughout the day.

It turns out that every plant cell keeps track of day and night independently of other cells, and that just three plant genes, called LCY, CCA1, and TOC1, are the clock’s master timekeepers. Such regulatory genes, and the regulatory proteins that they encode, are often controlled by external factors (e.g., nutrient availability or sunlight) in order to allow organisms to adjust their gene expression.

For example, regulatory proteins controlling the circadian clock in plants coordinate circadian activity as follows. TOC1 promotes the expression of LCY and CCA1, whereas LCY and CCA1 repress the expression of TOC1, resulting in a negative feedback loop. In the morning, sunlight activates the transcription of LCY and CCA1, triggering the repression of TOC1 transcription. As light diminishes, so does the production of LCY and CCA1, which in turn do not repress TOC1 any more. Transcription of TOC1 peaks at night and starts promoting the transcription of LCY and CCA1, which in turn repress the transcription of TOC1, and the cycle begins again.

LCY, CCA1, and TOC1 are able to control the transcription of other genes because the regulatory proteins that they encode are transcription factors, or master regulatory proteins that turn other genes on and off. A transcription factor regulates a gene by binding to a specific short DNA interval called a regulatory motif, or transcription factor binding site, in the gene's upstream region, a 600-1000 nucleotide-long region preceding the start of the gene. For example, CCA1 binds to AAAAAATCT in the upstream region of many genes regulated by CCA1.

The life of a bioinformatician would be easy if regulatory motifs were completely conserved, but the reality is more complex, as regulatory motifs may vary at some positions, e.g., CCA1 may instead bind to AAGAACTCT. But how can we locate these regulatory motifs without knowing what they look like in advance? We need to develop algorithms for motif finding, the problem of discovering a “hidden message” shared by a collection of strings.

### Motif Enumeration 

#A brute force algorithm for motif finding

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.

Implanted Motif Problem: Find all (k, d)-motifs in a collection of strings.
     Input: A collection of strings Dna, and integers k and d.
     Output: All (k, d)-motifs in Dna.

Brute force (also known as exhaustive search) is a general problem-solving technique that explores all possible solution candidates and checks whether each candidate solves the problem. Such algorithms require little effort to design and are guaranteed to produce a correct solution, but they may take an enormous amount of time, and the number of candidates may be too large to check.

A brute force approach for solving the Implanted Motif Problem is based on the observation that any (k, d)-motif must be at most d mismatches apart from some k-mer appearing in one of the strings of Dna. Therefore, we can generate all such k-mers and then check which of them are (k, d)-motifs. If you have forgotten how to generate these k-mers, take a look at CHARGING STATION:Generating the Neighborhood of a String.

In [231]:
def hammingDistance(x,y):
    nmm = 0
    for i in xrange(len(x)):
        if x[i] != y[i]:
            nmm += 1
    return nmm

In [2]:
#Approximate matching
def approximatePatternMatching(p, t, maxDistance):
    """Boolean function that returns whether or not pattern appears in text with d mismatches"""
    occurences = False
    for i in xrange(len(t) - len(p) + 1): #loop through every position in t where p could start
        nmm = 0
        match = True
        for j in xrange(len(p)): #Loop over characters
            if t[i+j] != p[j]: #Compare characters
                nmm += 1 #mismatch
                if nmm > maxDistance: #exceed max hamming distance
                    break
        if nmm <= maxDistance:
            occurences = True #approximate match
            break
    return occurences 

In [3]:
p = 'AAAA'
t = 'AGCCTGATCGTATGAAT'
approximatePatternMatching(p, t, 1)

False

In [4]:
def neighbors(pattern, d):
    '''Neighbors generates all k-mers of Hamming distance at most d from Pattern.'''
    if d == 0:
        return {pattern}
    if len(pattern) == 0:
        return {}
    if len(pattern) == 1:
        return {'A', 'C', 'G', 'T'}
    neighborhood = set()
    suffixneighbors = neighbors(pattern[1:], d)
    for text in suffixneighbors:
        if hammingDistance(text, pattern[1:]) < d:
            for nt in ['A', 'C', 'G', 'T']:
                neighborhood.add(nt + text)
        else:
            neighborhood.add(pattern[0] + text)
    return neighborhood     

In [5]:
def occurInAll(dna, pattern, d):
    occurence = True
    for string in dna:
        if approximatePatternMatching(pattern, string, d) == False:
            occurence = False
            break
    return occurence

In [6]:
dna = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
occurInAll(dna, 'TTT', 1)

True

In [11]:
def motifEnumeration(dna, k, d):
    '''Input: Integers k and d, followed by a collection of strings dna.
       Output: All (k, d)-motifs in dna.'''
    patterns = set()
    neighborhood = set()
    for string in dna:
        for i in range(len(string) -k + 1):
            pattern = string[i:i+k]
            neighborhood = neighbors(pattern,d)
            for pattern in neighborhood:
                if occurInAll(dna, pattern, d):
                    patterns.add(pattern)
    return patterns

In [12]:
dna = ['ATTTGGC', 'TGCCTTA', 'CGGTATC', 'GAAAATT']
x = motifEnumeration(dna, 3, 1)
' '.join(str(i) for i in x)

'ATT TTT GTT ATA'

### Motif finding problem

We can construct the 4 × k count matrix Count(Motifs) counting the number of occurrences of each nucleotide in each column of the motif matrix; the (i, j)-th element of Count(Motifs) stores the number of times that nucleotide i appears in column j of Motifs. We will further divide all of the elements in the count matrix by t, the number of rows in Motifs. This results in a profile matrix P = Profile(Motifs) for which Pi,j is the frequency of the i-th nucleotide in the j-th column of the motif matrix. Note that the elements of any column of the profile matrix sum to 1. The figure below shows the motif, count, and profile matrices for the NF-κB binding sites.

Motifs 

              T   C   G   G   G   G   g   T   T   T   t   t           
              c   C   G   G   t   G   A   c   T   T   a   C
              a   C   G   G   G   G   A   T   T   T   t   C
              T   t   G   G   G   G   A   c   T   T   t   t
              a   a   G   G   G   G   A   c   T   T   C   C
              T   t   G   G   G   G   A   c   T   T   C   C
              T   C   G   G   G   G   A   T   T   c   a   t
              T   C   G   G   G   G   A   T   T   c   C   t
              T   a   G   G   G   G   A   a   c   T   a   C
              T   C   G   G   G   t   A   T   a   a   C   C

Score

              3 + 4 + 0 + 0 + 1 + 1 + 1 + 5 + 2 + 3 + 6 + 4 = 30     

Count 

         A:   2   2   0   0   0   0   9   1   1   1   3   0          
         C:   1   6   0   0   0   0   0   4   1   2   4   6  
         G:   0   0  10  10   9   9   1   0   0   0   0   0  
         T:   7   2   0   0   1   1   0   5   8   7   3   4  

Profile

          A:  .2  .2   0   0   0   0  .9  .1  .1  .1  .3   0            
          C:  .1  .6   0   0   0   0   0  .4  .1  .2  .4  .6  
          G:   0   0   1   1  .9  .9  .1   0   0   0   0   0  
          T:  .7  .2   0   0  .1  .1   0  .5  .8  .7  .3  .4  
          
Consensus        
 
               T   C   G   G   G   G   A   T   T   T   C   C    

Entropy is a measure of the uncertainty of a probability distribution ($p_1, …, p_N$), and is defined as follows:

<b> Entropy </b> <br />
$H_(p_1,…,p_N) = − \displaystyle\sum_{i=1}^N p_i log_2 p_i$



The entropy of the completely conserved third column is 0, which is the minimum possible entropy. On the other hand, a column with equally-likely nucleotides (all probabilities equal to 1/4) has maximum possible entropy −4 · 1/4 · log2(1/4) = 2. In general, the more conserved the column, the smaller its entropy. Thus, entropy offers an improved method of scoring motif matrices: the entropy of a motif matrix is defined as the sum of the entropies of its columns. In this book, we will continue to use Score(Motifs) for simplicity, but the entropy score is used more often in practice.

Now that we have a good grasp of scoring a collection of k-mers, we are ready to formulate the Motif Finding Problem.

Motif Finding Problem: Given a collection of strings, find a set of k-mers, one from each string, that minimizes the score of the resulting motif.
     Input: A collection of strings Dna and an integer k.
     Output: A collection Motifs of k-mers, one from each string in Dna, minimizing Score(Motifs) among
     all possible choices of k-mers.

A brute force algorithm for the Motif Finding Problem (referred to as BRUTEFORCEMOTIFSEARCH) considers every possible choice of k-mers Motifs from Dna (one k-mer from each string of n nucleotides) and returns the collection Motifs having minimum score. Because there are n - k + 1 choices of k-mers in each of t sequences, there are (n - k + 1)^t different ways to form Motifs. For each choice of Motifs, the algorithm calculates Score(Motifs), which requires k · t steps. Thus, assuming that k is smaller than n, the overall running time of the algorithm is O(nt · k · t). We need to come up with a faster algorithm!

In [232]:
def distanceBetwnPatternAndStrings(pattern, dna):
    """  Input: A string Pattern followed by a collection of strings Dna.
     Output: d(Pattern, Dna)."""
    k = len(pattern)
    dist = 0
    for string in dna:
        hammingDist = float('Inf')
        for i in range(len(string) -k + 1):
            kmer = string[i:i+k]
            hDist = hammingDistance(pattern, kmer)
            if hammingDist > hDist:
                hammingDist = hDist
        dist += hammingDist
    return dist

In [16]:
dna = ['TTACCTTAAC', 'GATATCTGTC', 'ACGGCGTTCG', 'CCCTAAAGAG', 'CGTCAGAGGT']
distanceBetwnPatternAndStrings('AAA', dna)

5

In [17]:
def readFile(filename):
    f = open(filename)
    for line in f:
        line = line.rstrip()
        string = line.split()
    return string

In [76]:
dna = readFile('dist.txt')
dna

['GCCCCGACGTCGGTATGCTTGCGATTGGCCCGCCTGCGAGTCAAGGTGAAGTCTAGACAACATATATCGCGACAGGCTGGATCAGTGCAGACCTCAGGTGACCCCCGT',
 'AGCGTAAGGAACTATGTCGCCTAAGAGGCGTCTGGTTGACCTGACTACTATAGGATGTATAACCGCCATACGAGTAAGATTACCCAGAGATCCATATCTGATTGCATC',
 'AACCCTGGCTTCGTGCCGCCGACTTCCTCAATGGTGGAGGAATCAAGATATCAGTTTCCATTGATGATTGCTGTCACGGCCATCTCTGTGGTAGTGGACTAGCCACTC',
 'GTATGGAAGCTAGCGCCTCTTATAAACGATAAACTAAACATATAAGATCAGCCCCGGTTACAATAACGTAATATGGGTCGACCTTCAAGAACTGATACGGTCCGATGA',
 'GCTGCGACATCCTAATACCCCTTCGCCGCGAGTGGACTCCAAAAGCGCAGTAGAGACAGTCTTAAGGTCACGACTTCTCCTTATCGCGTGCAAAGCCACCCGATAACA',
 'TTCATTAACGGCCTTGATGAAAGTTCATGATTGTCGAATTGGCTTGCTCAGCTAATTATGGGCCGAGCGCCGGGCACCCACCGCAGATGGGGAATCTGGGTGCAGCTC',
 'GGGAACGTCGTAACCGGTAGTATTGTGTCGTACCACAGCCCTTATCAGAAAGGGAAGCCTCTGAATGAAGGCACCACGGAGTGCTTCGACTCACACCCGGACCCAAAT',
 'AGTCTGTGTGGACATGAGATCAACTGTGCAATGACGCACACCGGTGAGTGAAATGCATTTACTTTGGAAGGAATAACAGGAAGTTGCTAGCCTGTGTGAGATAAAAGA',
 'GTGCGTCATTCTCGACTGCGATGAGAGTGTCTCTGCTATAAACACCTGACAGCATTTTCACCTCGGCTTGCCGGATGTGAATAAAGGGGAGGTT

In [48]:
distanceBetwnPatternAndStrings('GGCTTAG', dna)

68

In [21]:
def patternToNumber(Pattern):
    symbolToNumber = {'A':0, 'C':1, 'G':2, 'T':3}
    n = len(Pattern)
    if n == 0:
        return 0
    elif n == 1:
        return symbolToNumber[Pattern]
    else:
        return 4*patternToNumber(Pattern[:-1]) + symbolToNumber[Pattern[-1]]

In [235]:
def numberToPattern (number, length):
    letter_value = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
    sequence = ''
    for i in range(length):
        ## take the remainder after dividing by 4, convert it to a base and prepend to sequence
        sequence = letter_value[number % 4] + sequence
        ## now do a floor division to get ready for the next base
        number = number // 4
    return sequence

In [240]:
def medianString(dna, k):
    """ Input: An integer k, followed by a collection of strings Dna.
     Output: A k-mer Pattern that minimizes d(Pattern, Dna) among all k-mers Pattern. (If there are
     multiple such strings Pattern, then you may return any one.)"""
    
    dist = float('Inf')
    median = None
    for i in range(4**k):
        pattern = numberToPattern(i,k)
        distance = distanceBetwnPatternAndStrings(pattern, dna)
        if dist > distance:
            dist = distance
            median = pattern
    return median

In [237]:
dna = ['AAATTGACGCAT', 'GACGACCACGTT', 'CGTCAGCGCCTG', 'GCTGAGCACCGG', 'AGTTCGGGACAG']
medianString(dna, 3)

'GAC'

In [25]:
dna = [line.strip() for line in open('rosalind_ba2b.txt', 'r')]
dna

['TGCTCAATGCGCGGGCTTTGTAAGCTAGGAATCAATAGCTCG',
 'AGCACGGATCTTGGAAAGACGTAAGCTTCGTCTGATGTGCTT',
 'GACATCTCGGCTGCGCTTCGACCACCGTGCTGAAAGCCCACC',
 'CCATAACTACTAGTAGAATCCTCTTGAGAATCAGGTGCGCTT',
 'AAAGAAGCGCTTTGTGAATCACCTGATTGAAAACGACTGACG',
 'GCCGAAAGAGTTATTGAGTTCGAAACTGGCGGGCTTTCATAA',
 'TATGATACCAGCGGGCCCCGAGTTGGTTGCGAGCTTTTGCTA',
 'GTACCCGATAGAAGCAGAGTACCAACTCCATGCGGCGCGCTT',
 'ATTACTAAACCAGACGTTTCTTGCGGAAAGGGGCTTGTGTGA',
 'CCTGCGGATTGGGGGCTTCCCTAAAATTTCAACGAAACTGCA']

In [26]:
medianString(dna, 6)

'GCGCTT'

In [79]:
dna = [line.strip() for line in open('median_string.txt', 'r')]
dna

['ATACATGAGAGAATGCTGCGTTTTGCTAAAGCTCAATCGTAA',
 'GAATGGGCTAACCACACTATGATGTTGACGTCTTGGACCATT',
 'CAATGCCGCGCCGGGAAGTTATAGATTCGTATGCTGCAAGTC',
 'ATGATGGTCCAATAGGTCCCGTTCGACAAAGATCAGCATGCC',
 'ATGATGCAACCATCGTCTCTACAGGACCGGAACCTAATAACT',
 'GTGTAAATGGTGACCTTCATATAATACTTGGCAAGGCACCAA',
 'GGAAAAAGAAACACGATAATGTTGGACCAACTGCCCGTTGAG',
 'ATGTTGAATACTAACTGATAGTTCCCGTCCCTATAGTCTTAT',
 'GCAGGGGTCGCATGCTTTACTCATCGAGTATAAAGAATGATG',
 'TAGGTCGCCGGCATGTTTTTCCAGATGCTGCTGAAAACCCCT']

In [80]:
medianString(dna, 6)

'ATGATG'

In [242]:
dna = ['CTCGATGAGTAGGAAAGTAGTTTCACTGGGCGAACCACCCCGGCGCTAATCCTAGTGCCC',
'GCAATCCTACCCGAGGCCACATATCAGTAGGAACTAGAACCACCACGGGTGGCTAGTTTC',
'GGTGTTGAACCACGGGGTTAGTTTCATCTATTGTAGGAATCGGCTTCAAATCCTACACAG']
medianString(dna, 7)

['AAAAAAA',
 'AAAAAAC',
 'AAAAAAG',
 'AAAAACC',
 'AAAACCA',
 'AAACACC',
 'AAACCAC',
 'AAATCCT',
 'AACCACC',
 'AATCCTA']

Why have we reformulated the Motif Finding Problem?

To see why we reformulated the Motif Finding Problem as the equivalent Median String Problem, consider the runtime of MEDIANSTRING and BRUTEFORCEMOTIFSEARCH. The former algorithm computes d(Pattern, Dna) for each of the 4k k-mers Pattern. Each computation of d(Pattern, Dna) requires a single pass over each string in Dna, which requires approximately k · n · t operations for t strings of length n in Dna. Therefore, MEDIANSTRING has a running time of $O(4^k · n · k · t)$, which in practice compares favorably with the $O(n^t · k · t)$ running time of BRUTEFORCEMOTIFSEARCH because the length of a motif (k) typically does not exceed 20 nucleotides, whereas t is measured in the thousands.

The Median String Problem teaches an important lesson, which is that sometimes rethinking how a problem is formulated can lead to dramatic improvements in the runtime required to solve it. In this case, our simple observation that Score(Motifs) could just as easily be computed row-by-row as column-by-column produced the faster MEDIANSTRING algorithm.

Of course, the ultimate test of a bioinformatics algorithm is how it performs in practice. Unfortunately, since MEDIANSTRING has to consider 4k k-mers, it becomes too slow for the Subtle Motif Problem, for which k = 15. We will run MEDIANSTRING with k = 13 in the hope that it will capture a substring of the correct 15-mer motif. The algorithm still requires half a day to run on our computer and returns the median string AAAAAtAGaGGGG (with distance 29). This 13-mer is not a substring of the implanted pattern AAAAAAAAGGGGGGG. but it does come close.

STOP and Think: How can a slightly incorrect median string of length 13 help us find the correct median string of length 15?

We have thus far assumed that the value of k is known in advance, which is not the case in practice. As a result, we are forced to run our motif finding algorithms for different values of k and then try to deduce the correct motif length. Since some regulatory motifs are rather long — later in the chapter, we will search for a biologically important motif of length 20 — MEDIANSTRING may be too slow to find them.

### Greedy motif search

In [3]:
def profileMostProbablekmer(text, k , matrix):
    '''Input: A string Text, an integer k, and a 4 × k matrix Profile.
     Output: A Profile-most probable k-mer in Text.'''
    maxp = None
    probablekmer = None
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        pt = 1
        for j in range(k):
            p = matrix[kmer[j]][j]
            pt *=p
        if maxp == None or pt > maxp:
            maxp = pt
            probablekmer = kmer
    return probablekmer

In [101]:
matrix = {'A':[0.2,0.2,0.3,0.2,0.3], 'C':[0.4,0.3,0.1,0.5,0.1], 'G':[0.3,0.3,0.5,0.2,0.4], 'T':[0.1,0.2,0.1,0.1,0.2]}
text = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k =  5
profileMostProbablekmer(text, k , matrix)

'CCGAG'

In [248]:
matrix = {'A':[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.1,0.2,0.3,0.4,0.5], 'C':[0.3,0.2,0.1,0.1,0.2,0.1,0.1,0.4,0.3,0.2,0.2,0.1], 'G':[0.2,0.1,0.4,0.3,0.1,0.1,0.1,0.3,0.1,0.1,0.2,0.1], 'T':[0.3,0.4,0.1,0.1,0.1,0.1,0.0,0.2,0.4,0.4,0.2,0.3]}
text = 'TTACCATGGGACCGCTGACTGATTTCTGGCGTCAGCGTGATGCTGGTGTGGATGACATTCCGGTGCGCTTTGTAAGCAGAGTTTA'
k =  12
profileMostProbablekmer(text, k , matrix)

'AAGCAGAGTTTA'

In [28]:
matrix = {'A':(0.28, 0.36, 0.12, 0.24, 0.32, 0.2, 0.32, 0.28), 
          'C':(0.24, 0.24, 0.32, 0.2, 0.24, 0.16, 0.32, 0.24), 
          'G':(0.28, 0.24, 0.12, 0.2, 0.16, 0.36, 0.24, 0.32), 
          'T':(0.2, 0.16, 0.44, 0.36, 0.28, 0.28, 0.12, 0.16)}
text = 'CGGCGTGCCGTAAGTATCCCCTGGTCGATTATGGAGGATACTTAAAAAGCCCAATCTTTCTATCGAGGGGTTTATGCGCAGCGCTGTCTTAGGGGCGCGTATAGAGTGGTAAGTAGTAACCTTTGGTGGGGACGAATAATTTGTTTAAAGCATGTATGAACATCAGGCAAATCGGAGAGCTGCGCGGCGCTGGGTAAAGG'
k =  8
profileMostProbablekmer(text, k , matrix)

'GATTATGG'

motifs = ['GGC', 'TTC', 'AAG', 'CTC']
x = []
for i in range(len(motifs[0])):
    y = ''
    for j in range(len(motifs)):
        y += motifs[j][i]
    x.append(y)
print x

motifs = ['GCG','AAG','AAG','ACG','CAA']
z = zip(*motifs)
z

motifs = ['GGC', 'TTC', 'AAG', 'CTC']
z = zip(*motifs)
print z
#z = ['GTAC', 'GTAT', 'CCGC']
d = {}
n = float(len(motifs))
print n
for i in range(len(z)):
    d.setdefault('A', []).append(z[i].count('A')/n)
    d.setdefault('C', []).append(z[i].count('C')/n)
    d.setdefault('G', []).append(z[i].count('G')/n)
    d.setdefault('T', []).append(z[i].count('T')/n)
print d 

In [243]:
def motifsToProfile(motifs):
    '''Given a list of motifs, the functions converts it into dictionary containing frequency profile of each nucleotide.'''
    z = zip(*motifs)
    d = {}
    n = float(len(motifs))
    for i in range(len(z)):
        d.setdefault('A', []).append(z[i].count('A')/n)
        d.setdefault('C', []).append(z[i].count('C')/n)
        d.setdefault('G', []).append(z[i].count('G')/n)
        d.setdefault('T', []).append(z[i].count('T')/n)
    return d

In [245]:
motifs = ['GTC', 'CCC', 'ATA', 'GCT']
motifsToProfile(motifs)

{'A': [0.25, 0.0, 0.25],
 'C': [0.25, 0.5, 0.5],
 'G': [0.5, 0.0, 0.0],
 'T': [0.0, 0.5, 0.25]}

def scoreMotifs(motifs):
    '''This function computes the score of list of motifs'''
    z = []
    for i in range(len(motifs[0])):
        y = ''
        for j in range(len(motifs)):
            y += motifs[j][i]
        z.append(y)
    totalscore = 0
    for string in z:
        score = len(string)-max([string.count('A'),string.count('C'), string.count('G'), string.count('T')])
        totalscore += score
    return totalscore  

In [6]:
def scoreMotifs(motifs):
    '''This function computes the score of list of motifs'''
    z = zip(*motifs)
    totalscore = 0
    for string in z:
        score = len(string)-max([string.count('A'),string.count('C'), string.count('G'), string.count('T')])
        totalscore += score
    return totalscore 

In [118]:
motifs = ['GCG','AAG','AAG','ACG','CAA']
scoreMotifs(motifs)

5

In [34]:
def greedyMotifSearch(dna, k, t):
    '''Input: Integers k and t, followed by a collection of strings Dna.
    Output: A collection of strings BestMotifs resulting from applying GREEDYMOTIFSEARCH(Dna,k,t).
    If at any step you find more than one Profile-most probable k-mer in a given string, use the
    one occurring first.'''
    bestMotifs = [i[:k] for i in dna]
    for i in range(len(dna[0])-k+1):
        motifs = [dna[0][i:i+k]]
        for j in range(1,t):
            profile = motifsToProfile(motifs)
            kmer = profileMostProbablekmer(dna[j], k , profile)
            motifs.append(kmer)
        if scoreMotifs(motifs) < scoreMotifs(bestMotifs):
            bestMotifs = motifs
    return bestMotifs

In [120]:
dna = ['GGCGTTCAGGCA','AAGAATCAGTCA','CAAGGAGTTCGC','CACGTCAATCAC','CAATAATATTCG']
greedyMotifSearch(dna, 3, 5)

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']

In [35]:
dna = [line.strip() for line in open('rosalind_ba2d.txt', 'r')]
x = greedyMotifSearch(dna, 12, 25)
' '.join(str(i) for i in x)

'AGCAAATAGAGG CCAAAGATGGGC GGCCGACCTACC TAAGCGTCGTGA AGTCAGTGGGGC TGAGAGAGTGGC CGTGCATCTTGG CGTGCATCGTGG TGTGGATGTTCC GGAGCGTTTGGC TGTGCATGTTCA CGACGACTTTGC AATCCATATAGC TTCCTGCCAATC CATGAACATAGC GATGCATCTACA TGCGAATGTAGC GGTGCATGTTTA TACCCGTTTAGC AGTGCATGGTTC TGTGCGACTGCC TGTGCATCTTTG TGCGCGAGGGGG AGTGCATGATGA GGTGCATCATTA'

In [179]:
dna = [line.strip() for line in open('greedyMotif.txt', 'r')]
x = greedyMotifSearch(dna, 12, 25)
' '.join(str(i) for i in x)

'TAACTTTCGACG ACGGTAAGTCAA CCGACCTCGATA CCCGGTGACCCG TCGCCTTCGCAG TCGGCAGATAAG AAGGCATCTAAG TAACCTTCGCAG GGTAAGTCATAC AAAGTTTCTTCG TAACCTTCGAAG CAGGTTTCAAAA CGGCCTTCGAAG TACCATTCGAAC TAGCTGTCACTG TCGCATTCGTCC TAGCACGCTAAG CCGACTTAGTAA GCGCCTTCGTAC TCAACAGCATAG TAGGCATATATG TGGACTTACTAA TCGAAATGGCCG ACGGTTGCTTAC TCGAATTCCTCA'

In [7]:
def motifsToProfileLaplace(motifs):
    '''Given a list of motifs, the functions converts it into dictionary containing frequency profile of each nucleotide.'''
    z = zip(*motifs)
    d = {}
    n = float(len(motifs))
    for i in range(len(z)):
        d.setdefault('A', []).append((z[i].count('A')+1)/(2*n))
        d.setdefault('C', []).append((z[i].count('C')+1)/(2*n))
        d.setdefault('G', []).append((z[i].count('G')+1)/(2*n))
        d.setdefault('T', []).append((z[i].count('T')+1)/(2*n))
    return d

In [247]:
motifs = ['GTC', 'CCC', 'ATA', 'GCT']
motifsToProfileLaplace(motifs)

{'A': [0.25, 0.125, 0.25],
 'C': [0.25, 0.375, 0.375],
 'G': [0.375, 0.125, 0.125],
 'T': [0.125, 0.375, 0.25]}

In [123]:
def greedyMotifSearchUsingLaplace(dna, k, t):
    '''Input: Integers k and t, followed by a collection of strings Dna.
     Output: A collection of strings BestMotifs resulting from applying GREEDYMOTIFSEARCH(Dna,k,t) with
     pseudocounts. If at any step you find more than one Profile-most probable k-mer in a given string,
     use the one occurring first.'''
    bestMotifs = [i[:k] for i in dna]
    for i in range(len(dna[0])-k+1):
        motifs = [dna[0][i:i+k]]
        for j in range(1,t):
            profile = motifsToProfileLaplace(motifs)
            kmer = profileMostProbablekmer(dna[j], k , profile)
            motifs.append(kmer)
        if scoreMotifs(motifs) < scoreMotifs(bestMotifs):
            bestMotifs = motifs
    return bestMotifs

In [124]:
dna = ['GGCGTTCAGGCA','AAGAATCAGTCA','CAAGGAGTTCGC','CACGTCAATCAC','CAATAATATTCG']
greedyMotifSearchUsingLaplace(dna, 3, 5)

['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

In [200]:
dna = [line.strip() for line in open('greedyMotif2.txt', 'r')]
x = greedyMotifSearchUsingLaplace(dna, 12, 25)
' '.join(str(i) for i in x)

'TATCGGGGTTCC GTTCGGAGTGCC TGTCGGCGTCCC ACTCGGAGTGCC AGTCGGGGTACC CTTCGGCGTCCC CGTCGGCGTCCC CATCGGTGTCCC AGTCGGTGTCCC ACTCGGGGTTCC TTTCGGCGTCCC TCTCGGTGTACC TCTCGGGGTGCC CTTCGGCGTGCC TATCGGTGTCCC CTTCGGCGTACC CGTCGGTGTTCC CATCGGCGTACC GGTCGGAGTACC GCTCGGTGTACC TGTCGGTGTTCC GATCGGCGTCCC ATTCGGCGTTCC TCTCGGCGTGCC CCTCGGTGTGCC'

In [249]:
def motifsFromProfile(dna, k , profile):
    '''Computes the profileMostProbablekmer for each text in dna and returns it as a list.'''
    return [profileMostProbablekmer(text, k , profile) for text in dna]

In [126]:
matrix = {'A':(0.2,0.2,0.3,0.2,0.3), 'C':(0.4,0.3,0.1,0.5,0.1), 'G':(0.3,0.3,0.5,0.2,0.4), 'T':(0.1,0.2,0.1,0.1,0.2)}
dna = ['ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT', 'ACCGCTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGAGCCC']
k =  5
motifsFromProfile(dna, k , matrix)

['CCGAG', 'CCGCT']

In [251]:
matrix = {'A': [0.25, 0.125, 0.25],
 'C': [0.25, 0.375, 0.375],
 'G': [0.375, 0.125, 0.125],
 'T': [0.125, 0.375, 0.25]}
dna = ['ATGAGGTC', 'GCCCTAGA', 'AAATAGAT', 'TTGTGCTA']
k =  3
motifsFromProfile(dna, k , matrix)

['GTC', 'GCC', 'ATA', 'GCT']

In [8]:
def random_kmers(dna, k):
    '''Returns a collection of randomly chosen k-mers in Dna. t is the number of items in dna list'''
    import random
    motifs = []
    for text in dna:
        i = random.randrange(0,len(dna[0])-k+1)
        motifs.append(text[i:i+k])
    return motifs

In [14]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
     'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
random_kmers(dna, 8)

['TTCAGTAA', 'GGTGCCAG', 'CGAGACCG', 'CAGGTGCA', 'AATCCACC']

In [55]:
def randomizedMotifSearch(dna, k):
    '''Input: A collection of strings dna followed by integer k.
     Output: BestMotif resulting from running randomizedMotifSearch(dna, k) one
     time. Remember to use pseudocounts!'''
    motifs = random_kmers(dna, k)
    bestMotifs = motifs
    while True:
        profile = motifsToProfileLaplace(motifs)
        motifs = motifsFromProfile(dna, k , profile)
        if scoreMotifs(motifs) < scoreMotifs(bestMotifs):
            bestMotifs = motifs
        else:
            return bestMotifs

dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
     'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
bestMotifs = None
bestScore = None
n = 0
while n <= 5000:
    n += 1
    motifs = randomizedMotifSearch(dna, 8)
    score = scoreMotifs(motifs)
    if bestScore == None or score <= bestScore:
        bestScore = score
        bestMotifs = motifs
print bestMotifs, bestScore

In [56]:
def looprandomizedMotifSearch(dna, k, nloop):
    '''Input: A collection of strings dna followed by integers k and nloop(number of times randomizedMotifSearch(dna, k) will be run.
     Output: A collection BestMotifs resulting from running randomizedMotifSearch(dna, k) 1,000
     times'''
    bestMotifs = None
    bestScore = None
    n = 0
    while n <= nloop:
        n += 1
        motifs = randomizedMotifSearch(dna, k)
        score = scoreMotifs(motifs)
        if bestScore is None or score < bestScore:
            bestScore = score
            bestMotifs = motifs
    return bestMotifs, bestScore

In [46]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
     'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
looprandomizedMotifSearch(dna, 8, 2000)

(['AACGGCCA', 'AAGTGCCA', 'TAGTACCG', 'AAGTTTCA', 'ACGTGCAA'], 9)

In [60]:
#Debug dataset 1
dna = ['AATTGGCACATCATTATCGATAACGATTCGCCGCATTGCC', 'GGTTAACATCGAATAACTGACACCTGCTCTGGCACCGCTC', 'AATTGGCGGCGGTATAGCCAGATAGTGCCAATAATTTCCT',
      'GGTTAATGGTGAAGTGTGGGTTATGGGGAAAGGCAGACTG', 'AATTGGACGGCAACTACGGTTACAACGCAGCAAGAATATT', 'GGTTAACTGTTGTTGCTAACACCGTTAAGCGACGGCAACT',
      'AATTGGCCAACGTAGGCGCGGCTTGGCATCTCGGTGTGTG', 'GGTTAAAAGGCGCATCTTACTCTTTTCGCTTTCAAAAAAA']
looprandomizedMotifSearch(dna, 6, 2000)

(['CGATAA',
  'GGTTAA',
  'GGTATA',
  'GGTTAA',
  'GGTTAC',
  'GGTTAA',
  'GGCCAA',
  'GGTTAA'],
 7)

In [49]:
#Debug dataset 1 ans
motifs = ['CGATAA', 'GGTTAA', 'GGTATA', 'GGTTAA', 'GGTTAC', 'GGTTAA', 'GGCCAA', 'GGTTAA']
scoreMotifs(motifs)

7

In [61]:
#Debug dataset 2
dna = ['GCACATCATTAAACGATTCGCCGCATTGCCTCGATTAACC', 'TCATAACTGACACCTGCTCTGGCACCGCTCATCCAAGGCC', 'AAGCGGGTATAGCCAGATAGTGCCAATAATTTCCTTAACC',
      'AGTCGGTGGTGAAGTGTGGGTTATGGGGAAAGGCAAGGCC', 'AACCGGACGGCAACTACGGTTACAACGCAGCAAGTTAACC', 'AGGCGTCTGTTGTTGCTAACACCGTTAAGCGACGAAGGCC', 
      'AAGCTTCCAACATCGTCTTGGCATCTCGGTGTGTTTAACC', 'AATTGAACATCTTACTCTTTTCGCTTTCAAAAAAAAGGCC']
looprandomizedMotifSearch(dna, 6, 2000)

(['TTAACC',
  'ATAACT',
  'TTAACC',
  'TGAAGT',
  'TTAACC',
  'TTAAGC',
  'TTAACC',
  'TGAACA'],
 8)

In [62]:
#Debug dataset 2 ans
motifs = ['TTAACC', 'ATAACT', 'TTAACC', 'TGAAGT', 'TTAACC', 'TTAAGC', 'TTAACC', 'TGAACA']
scoreMotifs(motifs)

8

In [92]:
#ans
motifs = ['TCTCGGGG','CCAAGGTG','TACAGGCG','TTCAGGTG','TCCACGTG']
scoreMotifs(motifs)

9

In [3]:
dna = [line.strip() for line in open('dataset_161_5.txt', 'r')]
dna

['CTGCCAAAACGTTCGTTGGGTTTAGTAGGAGCCGAGGGCCACCTTTAGAACCTTCGCTGTTGAATAAGTTGCCAAGAAGCGCTATAGGCATTAGCTTTTGCCTGCGGGATCTCCAAGCGGGAATACTTTGCTACCAAAGTTAATACAATCGCCTGCTCAGAAACTGCCAAAACGTTCG',
 'TTGGGTTTAGTAGGAGCCGAGGGCCACCTTTAGAACCTTCGCTGTTGAATACTGAGCATGGTACGCAGTTGCCAAGAAGCGCTATAGGCATTAGCTTTTGCCTGCGGGATCTCCAAGCGGGAATACTTTGCTACCAAAGTTAATACAATCGCCTGCTCAGAAACTGCCAAAACGTTCG',
 'TGCTCACGTGAAACGACCCTTAGTGTAGTGGCCTACTCTGGCAATATCACGCCCACCTCTTCTAGTATGGTCCATGGAGAAAGCACATTGACCTTGGTTTATATTAAATGTTCATTTGCGTGATGGAGGGGCAGCCAGTTAGCCCAACGCTATTCCCTCCCATCTGGTTTATCTTCCA',
 'TAAGCCACATCAGTGCTGTCAGCGGTGGATATAGGCAATAGCAACCTCGTTAGTGCTTTATGCCAGCGGATTCAGCTGGCAGGTGTACGCAGGGGGATCATTCAGTGCCAGGTAATAGACCAACCCGGAATTAGGCACTCCGGCCTATCCCCCTAGCATGATCGTCAGCCTCCTATAA',
 'CCCGTATTTTAGATTCTATCCAGATTCTAGCTGACCAAGCCACGCGCGATAAATACCGGGTCAATAGTTAGTCTTAGCTTTTGTTTATTCTCGCCTCACCACTTAAAGTCCCTGTTAGCGCTCCGTCCGTCAGCAGTATGTCTGGCCGGGGTACGCAGCCAAATCAAAGGCCACTAAA',
 'CTTGCGCAAGCCACCTCTCGATACATTCTGAGCGCCGAACACGAGCGTAAGTGCCAGCTCTAGTCGCATGCCTGATAACATTA

In [37]:
bestMotifs,bestScore = looprandomizedMotifSearch(dna, 15, 2000)
' '.join(str(i) for i in bestMotifs)

'TGCTCCAGCCCCGCT TGACGGACCCGGGTT TGAACTGACCGGGTT TGAACAACAGTGGTT TGAAGGTCCCGGGTT GTTACAACCCGGGTT TATGCAACCCGGGTT TGAACAACCCACTTT TGATACACCCGGGTT CTAACAACCCGGGTG TGTCGAACCCGGGTT TGAACAACCCGAAGT AGAACAACCCGGGAC TGAACAACCCGGCAC TGAAATTCCCGGGTT TGAACAACCGAAGTT TGAACGGACCGGGTT TGAACACATCGGGTT TGAACAAGGGGGGTT TGAACAGGTCGGGTT'

In [38]:
bestScore

64

#EXERCISE BREAK: Compute the probability that ten randomly selected 15-mers from the ten 600-nucleotide long strings in the Subtle Motif Problem capture at least one implanted 15-mer. (Allowable error: 0.000001)

kmers = 600-15+1 = 586

P(capturing the 15-mer in string 1) = 1/586

P(not capturing the 15-mer in string 1) = 585/586

P(not capturing the 15-mer in 10 strings) = (585/586)**10 

probability (capturing at least one 15-mer) = 1- (585/586)**10 

In [64]:
#EXERCISE BREAK: Compute the probability that ten randomly selected 15-mers from ten 600-nucleotide long strings 
#(as in the Subtle Motif Problem) capture at least two implanted 15-mers. (Allowable error: 0.000001)

1- ((585.0/586)**9)*(1/586.0) - ((585.0/586)**10)

0.015253942906766849

### Gibbs Sampling

In [9]:
import random
def Random(p):
    wheel = [0]*(len(p)+1)
    #Normalization of probability distribution
    s = sum(p)
    if s!= float(1):
        p = [float(i)/sum(p) for i in p]
    #Make a roulette wheel
    for index in range(len(p)):
        wheel[index+1] = wheel[index] + p[index]
    #print wheel
    #Rolling a biased dice
    r = random.random() #Randomly generate a number between 0 and 1
    #print r
    for i in range(len(wheel)-1):
        if wheel[i] < r < wheel[i+1]:
            result = i
    return result

In [200]:
print Random([0.2,0.7,0.1, 0.2])

1


In [10]:
def profileRandomlyGeneratedkmer(text, k , matrix):
    '''Input: A string Text, an integer k, and a 4 × k matrix Profile.
     Output: A Profile-most probable k-mer in Text.'''
    prob = []
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        pt = 1
        for j in range(k):
            p = matrix[kmer[j]][j]
            pt *=p
        prob.append(pt)
    #print prob
    kmer_index = Random(prob)
    randomkmer = text[kmer_index:kmer_index+k]
    return randomkmer

In [172]:
matrix = {'A':[0.2,0.2,0.3,0.2,0.3], 'C':[0.4,0.3,0.1,0.5,0.1], 'G':[0.3,0.3,0.5,0.2,0.4], 'T':[0.1,0.2,0.1,0.1,0.2]}
text = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k =  5
profileRandomlyGeneratedkmer(text, k , matrix)

'CCGAG'

In [11]:
def gibbsSampler(dna, k, t, N):
    motifs = random_kmers(dna, k)
    bestMotifs = motifs
    for j in range(N):
        i = random.randrange(0,t)
        del motifs[i]
        profile = motifsToProfileLaplace(motifs)
        motifi = profileRandomlyGeneratedkmer(dna[i], k , profile)
        motifs.insert(i, motifi)
        if scoreMotifs(motifs) < scoreMotifs(bestMotifs):
            bestMotifs = motifs
    return bestMotifs

In [168]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
     'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
     'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
     'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
     'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
gibbsSampler(dna, 8, 5, 100)

['TCTCGGGG', 'CAAGGTGC', 'TACCGAGA', 'TCAGGTGC', 'CCAGCTCC']

In [12]:
def loopGibbsSampler(dna, k, t, N):
    bestMotifs = None
    bestScore = None
    n = 0
    while n <= 20:
        n += 1
        motifs = gibbsSampler(dna, k, t, N)
        score = scoreMotifs(motifs)
        if bestScore is None or score < bestScore:
            bestScore = score
            bestMotifs = motifs
    return bestMotifs, bestScore

In [13]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
     'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
     'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
     'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
     'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
loopGibbsSampler(dna, 8, 5, 100)

(['CAGTAACC', 'AAGGTGCC', 'CCGAGACC', 'CAGGTGCA', 'CACGTGCA'], 11)

In [137]:
#Sample ans
motifs = ['TCTCGGGG','CCAAGGTG','TACAGGCG','TTCAGGTG','TCCACGTG']
scoreMotifs(motifs)

9

In [14]:
dna = [line.strip() for line in open('dataset_163_4.txt', 'r')]
dna

['TAAGGCTCGATTCATCGCTGATATCAGTTATTTGGCTCACGGTTTCTGATTCCCCGGATGTGAAGCTAGGGAGACAGACCTTGGGAGTGATAGAACGTACTTAAACCCCTCGTCGCCAGTCGGCGCGATTGCTGCAGCGCTTAATAACTAAGTAAGACTAGCTACGCAAGTTTTGGCCTCCCATATCGAAAGTGAATAAGCTCAATGGATAGCTCCCCACGTAACGGGGCAGTCTTAACAGCGGAGCTTGCTCGCTTGCAGATCCGGTAGGAGTCAGTCATGGACTTCGTAAATCCGCGTCCGTCGGCAGGGCCTAAATAAGGCTCGATTCAT',
 'CGCTGATATCAGTTATTTGACCTGCAAGCTCTTAGCTCACGGTTTCTGATTCCCCGGATGTGAAGCTAGGGAGACAGACCTTGGGAGTGATAGAACGTACTTAAACCCCTCGTCGCCAGTCGGCGCGATTGCTGCAGCGCTTAATAACTAAGTAAGACTAGCTACGCAAGTTTTGGCCTCCCATATCGAAAGTGAATAAGCTCAATGGATAGCTCCCCACGTAACGGGGCAGTCTTAACAGCGGAGCTTGCTCGCTTGCAGATCCGGTAGGAGTCAGTCATGGACTTCGTAAATCCGCGTCCGTCGGCAGGGCCTAAATAAGGCTCGATTCAT',
 'CAGATGGCAGGGCCGCCAGTGTTGGTTCCAAGGAATATGCTAGGCTCGGATACAGTCGAAGTAGCCCTCTCCGGATGAACTCTTGAGTACTTATAAAGTATTTCCTCCAACCTTCTAGCTGGGCCCAACCCCACCGCTCTTACATGCCATTAAGAGGTCCGTAATGATAGCACTGAATGAAAGAAGCACTAAGTAGATATAGCGGCACTAATACAATGTTTCCTTTTTTGTGTCGATACACATAGCACTCATGTCCAGGCGCGGAATGGTCGAGTACAGTTTTACGCATAAGATCCCTTTATGAGCTTGGCGGTTATATTCG

In [15]:
bestMotifs,bestScore = loopGibbsSampler(dna, 15, 20, 2000)
' '.join(str(i) for i in bestMotifs)

'TCATCGCTGATATCA ACCTGCAAGCTCTTA ACCCCACCGCTCTTA ACCCCTGTGCTCTTA CGCCCGAAGCTCTTC CTTCCGAAGCTCTTA CCCCGCAAGTGCCAG ACCCCGAAATGCTTA TCCCCGAAGCTCTAT ATGTCGAAGCTCTTA ACCCCGAAGTAGTTA ACTAAGAAGCTCTTA ACCGTTAAGCTCTTA ACCCTATAGCTCTTA ACCCCGACCGTCTTA ACCCCGTTTCTCTTA ACCCCGAAGCCTGTA ACCCCGCGCCTCTTA ACCCCGAAGCTCGGG ACCCACCAGCTCTTA'

In [16]:
bestScore

70

In [1]:
x = 'TGCTCCAGCCCCGCT TGACGGACCCGGGTT TGAACTGACCGGGTT TGAACAACAGTGGTT TGAAGGTCCCGGGTT GTTACAACCCGGGTT TATGCAACCCGGGTT TGAACAACCCACTTT TGATACACCCGGGTT CTAACAACCCGGGTG TGTCGAACCCGGGTT TGAACAACCCGAAGT AGAACAACCCGGGAC TGAACAACCCGGCAC TGAAATTCCCGGGTT TGAACAACCGAAGTT TGAACGGACCGGGTT TGAACACATCGGGTT TGAACAAGGGGGGTT TGAACAGGTCGGGTT'

In [5]:
for word in x.split(): print word

TGCTCCAGCCCCGCT
TGACGGACCCGGGTT
TGAACTGACCGGGTT
TGAACAACAGTGGTT
TGAAGGTCCCGGGTT
GTTACAACCCGGGTT
TATGCAACCCGGGTT
TGAACAACCCACTTT
TGATACACCCGGGTT
CTAACAACCCGGGTG
TGTCGAACCCGGGTT
TGAACAACCCGAAGT
AGAACAACCCGGGAC
TGAACAACCCGGCAC
TGAAATTCCCGGGTT
TGAACAACCGAAGTT
TGAACGGACCGGGTT
TGAACACATCGGGTT
TGAACAAGGGGGGTT
TGAACAGGTCGGGTT
